In [1]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle
IMG_SIZE = 96
LR = 1e-3  #.001 learing rate

nb_classes=28

MODEL_NAME = 'handsign.model'

path='data'

IMG_SIZE = 96

def create_train_data():
    training_data = []
    label=0
    for (dirpath,dirnames,filenames) in os.walk(path):
        for dirname in dirnames:
            print(dirname)
            for(direcpath,direcnames,files) in os.walk(path+"/"+dirname):
                for file in files:
                        actual_path=path+"/"+dirname+"/"+file
                       
                        # label=label_img(dirname)
                        path1 =path+"/"+dirname+'/'+file
                        img=cv2.imread(path1,cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                        training_data.append([np.array(img),label])
            label=label+1
    shuffle(training_data)
    return training_data

train_data = create_train_data()
# test_data = np.load('test_data.npy',encoding="latin1")


down
leftagain
right
stop


In [2]:
train = train_data[:]
test = train_data[:100]

##print(train)

In [3]:
def one_hot_targets_(labels_dense,nb_classes):
    targets = np.array(Y).reshape(-1)
    print(targets)
    one_hot_targets = np.eye(nb_classes)[targets]
    return one_hot_targets

print('traindatlen:'+str(len(train)))
print('testdatalen:'+str(len(test)))



traindatlen:1872
testdatalen:100


In [4]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]
Y1=one_hot_targets_(Y,nb_classes)
# 
# print('max y'+str(max(Y)))
# print('min y'+str(min(Y)))
print('val y'+str(Y1))
print('len X:'+str(len(X)))
print('len Y:'+str(len(Y)))


[3 1 1 ... 3 0 0]
val y[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
len X:1872
len Y:1872


In [5]:
test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]
test_y1=one_hot_targets_(test_y,nb_classes)
test_y=test_y1
Y=Y1
print('test_x:'+str(len(test_x)))
print('test_y:'+str(len(test_y)))
print('val y'+str(test_y1))


[3 1 1 ... 3 0 0]
test_x:100
test_y:1872
val y[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [6]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [7]:
IMG_SIZE = 96
LR = 1e-3

nb_classes=28

MODEL_NAME = 'handsign.model'
def cnn_model():
	convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

	convnet = conv_2d(convnet, 8, 5, activation='relu')
	convnet = max_pool_2d(convnet, 5)

	convnet = conv_2d(convnet, 16, 5, activation='relu')
	convnet = max_pool_2d(convnet, 5)

	convnet = conv_2d(convnet, 32, 5, activation='relu')
	convnet = max_pool_2d(convnet, 5)

	convnet = conv_2d(convnet, 64, 5, activation='relu')
	convnet = max_pool_2d(convnet, 5)

	convnet = conv_2d(convnet, 128, 5, activation='relu')
	convnet = max_pool_2d(convnet, 5)

	convnet = fully_connected(convnet, 1024, activation='relu')
	convnet = dropout(convnet, 0.8)

	convnet = fully_connected(convnet, nb_classes, activation='softmax')
	convnet = regression(convnet, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='targets')

	model = tflearn.DNN(convnet, tensorboard_dir='log')
	
	return model

In [8]:
model=cnn_model()

model.fit({'input': X}, {'targets': Y}, n_epoch=15, validation_set=({'input': test_x}, {'targets': test_y}), 
snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

model.save(MODEL_NAME)

score = model.evaluate(test_x, test_y)
print('Test accuarcy: %0.4f%%' % (score[0] * 100))

Training Step: 449  | total loss: 0.04867 | time: 5.444s
| Adam | epoch: 015 | loss: 0.04867 - acc: 0.9900 -- iter: 1856/1872
Training Step: 450  | total loss: 0.05070 | time: 6.628s
| Adam | epoch: 015 | loss: 0.05070 - acc: 0.9895 | val_loss: 0.48710 - val_acc: 0.9500 -- iter: 1872/1872
--
INFO:tensorflow:C:\Users\hp\image_classification\handsign.model is not in all_model_checkpoint_paths. Manually adding it.
Test accuarcy: 95.0000%


In [ ]:
IMG_SIZE = 96
LR = 1e-3

nb_classes=28


# organize imports
import cv2
import imutils
import numpy as np

from collections import Counter

import time

out_label=['left','stop','down','right']


pre=[]

s=''
cchar=[0,0]
c1=''

# initialize weight for running average
aWeight = 0.5

# get the reference to the webcam
camera = cv2.VideoCapture(0)

# region of interest (ROI) coordinates
top, right, bottom, left = 170, 150, 425, 450

# initialize num of frames
num_frames = 0

flag=0
flag1=0

# keep looping, until interrupted
while(True):
    # get the current frame
    (grabbed, frame) = camera.read()

    # resize the frame
    frame = imutils.resize(frame, width=700)

    # flip the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)

    # clone the frame
    clone = frame.copy()

    # get the height and width of the frame
    (height, width) = frame.shape[:2]

    # get the ROI
    roi = frame[top:bottom, right:left]

    # convert the roi to grayscale and blur it
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    
    # cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
    
    img=gray
    # img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img=cv2.imread("240fn.jpg",cv2.IMREAD_GRAYSCALE)
    # img=cv2.cvtColor(bw_image,cv2.COLOR_BGR2GRAY)
    
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    test_data =img

    orig = img
    data = img.reshape(IMG_SIZE,IMG_SIZE,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    # print(model_out)
    model_out = model.predict([data])[0]
        # print(model_out)
    pnb=np.argmax(model_out)
    print(str(np.argmax(model_out))+" "+str(out_label[pnb]))

    pre.append(out_label[pnb]) 


    cv2.putText(clone,
           '%s ' % (str(out_label[pnb])),
           (450, 150), cv2.FONT_HERSHEY_PLAIN,5,(0, 255, 0))

            

            
        


    # draw the segmented hand
    cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

    cv2.putText(clone,
                   '%s ' % (str(s)),
                   (10, 60), cv2.FONT_HERSHEY_PLAIN,3,(0, 0, 0))

    # increment the number of frames
    num_frames += 1
    # time.sleep(.3)
    # display the frame with segmented hand
    cv2.imshow("Video Feed", clone)

    # observe the keypress by the user
    keypress = cv2.waitKey(1) & 0xFF

    # if the user pressed "q", then stop looping
    if keypress == ord("q"):
        break


# free up memory
camera.release()
cv2.destroyAllWindows()


0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
2 down
2 down
2 down
0 left
3 right
0 left
3 right
3 right
3 right
3 right
0 left
3 right
3 right
3 right
0 left
0 left
0 left
3 right
3 right
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
1 stop
0 left
0 left
0 left
1 stop
1 stop
1 stop
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 left
0 l